In [1]:
import pandas as pd
import math
import snappy
import sage

In [2]:
pd.set_option('display.max_colwidth', None)

### Table of Knot Invariants for $T(4,n)$
Knot signature, Alexander polynomial, Arf, upsilon, etc.

In [3]:
def symmetric_alexander_poly(K):
    alex = K.alexander_polynomial(norm=False)
    alex_sym = alex.shift(-min(alex.exponents()) // 2) 
    return alex_sym

In [4]:
T45 = snappy.Link('T(4,5)')
T45.alexander_polynomial()

t^12 - t^11 + t^8 - t^6 + t^4 - t + 1

In [5]:
symmetric_alexander_poly(T45)

t^-6 - t^-5 + t^-2 - 1 + t^2 - t^5 + t^6

In [6]:
# compute d_invariant for T(p,q) torus knots
def d_invariant(T_pq):
    alex_sym = symmetric_alexander_poly(T_pq)
    coeff_dict = alex_sym.dict()
    pos_exponents = [k for k in coeff_dict.keys() if k>0]
    
    d_T = 0
    for j in pos_exponents:
        d_T += j * coeff_dict[j]
        
    return 2 * d_T

In [10]:
def make_table(p, max_q=None):
    if max_q is None:
        max_q = p+12
        
    invariants = []
    for q in range(p+1, max_q+1):
        if math.gcd(p,q) != 1: continue
    
        knot_name = f'T({p},{q})'
        knot = snappy.Link(knot_name)
        sage_knot = knot.sage_link()
    
        floer = knot.knot_floer_homology()
        #print(floer)
        
        invariants.append([q, knot_name, knot.signature(), sage_knot.arf_invariant(), floer['nu'], d_invariant(knot)])
        
    return pd.DataFrame(data=invariants, columns=['q', 'Knot', '$\\sigma$', 'Arf', '$\\nu$', 'd'])

In [20]:
df = make_table(4, max_q=25)
df

,q,Knot,$\sigma$,Arf,$\nu$,d
0,5,"T(4,5)",-8,1,6,6
1,7,"T(4,7)",-14,0,9,8
2,9,"T(4,9)",-16,0,12,8
3,11,"T(4,11)",-22,1,15,10
4,13,"T(4,13)",-24,1,18,14
5,15,"T(4,15)",-30,0,21,16
6,17,"T(4,17)",-32,0,24,16
7,19,"T(4,19)",-38,1,27,18
8,21,"T(4,21)",-40,1,30,22
9,23,"T(4,23)",-46,0,33,24


Observe that Remark 2.2 from Fairchild, et al. holds:

For odd $q$,
$$
\text{Arf}(T(p,q)) =
\begin{cases}
0 & p \text{ odd or } q\equiv\pm1\bmod 8, \\
1 & p \text { even and } q \equiv\pm 3\bmod 8.
\end{cases}
$$

Some data for verifying that `d_invariant` is working properly.

Check these against Proposition 3.9 from Fairchild, et al.

In [12]:
make_table(5, max_q=11)

,q,Knot,$\sigma$,Arf,$\nu$,d
0,6,"T(5,6)",-16,1,10,-6
1,7,"T(5,7)",-16,0,12,-8
2,8,"T(5,8)",-20,1,14,-10
3,9,"T(5,9)",-24,0,16,-12
4,11,"T(5,11)",-24,0,20,-12


In [19]:
make_table(6, max_q=15)

,q,Knot,$\sigma$,Arf,$\nu$,d
0,7,"T(6,7)",-18,0,15,12
1,11,"T(6,11)",-34,1,25,18
2,13,"T(6,13)",-36,1,30,18


## Lower bounds on $\gamma_4(T(4,n))$

Let's see for which $n$ does $\gamma_4(T(4,n))\ge 2$ using Proposition 2.3 from Fairchild, et al.

Proposition 2.3. Given a knot $K$ in $S^3$, if $\sigma(K)+4\text{Arf}(K)\equiv 4\bmod 8$ then $\gamma_4(K)\ge2$.

In [28]:
sig_arf = df['$\\sigma$'] + 4 * df['Arf']
sig_arf_lb_cond = sig_arf % 8
lb_applies = ['✓' if x == 4 else ' ' for x in sig_arf_lb_cond]
pd.DataFrame(data=zip(df['q'], sig_arf, sig_arf_lb_cond, lb_applies),
             columns=['q', '$\\sigma(K)+4\\text{Arf}(K)$', '$\\sigma(K)+4\\text{Arf}(K)\\bmod8$', 'Lower bound applies?'])

,q,$\sigma(K)+4\text{Arf}(K)$,$\sigma(K)+4\text{Arf}(K)\bmod8$,Lower bound applies?
0,5,-4,4,✓
1,7,-14,2,
2,9,-16,0,
3,11,-18,6,
4,13,-20,4,✓
5,15,-30,2,
6,17,-32,0,
7,19,-34,6,
8,21,-36,4,✓
9,23,-46,2,


### TODO - look into fixing column header latex formatting

https://stackoverflow.com/questions/72027874/python-formatting-a-pandas-dataframe-head-with-latex

Now let's compute the lower bound in Theorem 2.6, again from Fairchild, et al.

For a knot $K$ in $S^3$,
$$
\left| \nu(K)+\frac{\sigma(K)}{2} \right| \le \gamma_4(K).
$$

In [22]:
lb_nu = abs(df['$\\nu$'] + 0.5 * df['$\\sigma$'])

In [23]:
pd.DataFrame(data=zip(df['q'], lb_nu),
             columns=['q', '$\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|$'])
#.style.set_table_styles([dict(selector="th", props=[('max-width', '120px'),
#                                ('text-overflow', 'ellipsis'), ('overflow', 'hidden')])])

,q,$\left| \nu(K)+\frac{\sigma(K)}{2} \right|$
0,5,2.0
1,7,2.0
2,9,4.0
3,11,4.0
4,13,6.0
5,15,6.0
6,17,8.0
7,19,8.0
8,21,10.0
9,23,10.0


Lastly, let's compute the lower bound from Theorem 2.7 using the $d$-invariant.

For a knot $K$,
$$
\frac{\sigma(K)}{2} - d(K) \le \gamma_4(K).
$$

In [24]:
lb_d = 0.5 * df['$\\sigma$'] - df['d']

In [25]:
pd.DataFrame(data=zip(df['q'], lb_d),
             columns=['q', '$\\frac{\\sigma(K)}{2} - d(K) \\le \\gamma_4(K)$'])

,q,$\frac{\sigma(K)}{2} - d(K) \le \gamma_4(K)$
0,5,-10.0
1,7,-15.0
2,9,-16.0
3,11,-21.0
4,13,-26.0
5,15,-31.0
6,17,-32.0
7,19,-37.0
8,21,-42.0
9,23,-47.0


## Putting it all together

In [30]:
sig_arf_lb = [2 if x == 4 else 0 for x in sig_arf_lb_cond]

In [64]:
max_lbs = [max(x) for x in zip(sig_arf_lb, lb_nu, lb_d)]

In [66]:
pd.DataFrame(zip(df['q'], df['Knot'], sig_arf_lb, lb_nu, lb_d, max_lbs),
             columns=['q', 'Knot', 'arf lb', 'nu lb', 'd lb', 'maximum lb'])

,q,Knot,arf lb,nu lb,d lb,maximum lb
0,5,"T(4,5)",2,2.0,-10.0,2.0
1,7,"T(4,7)",0,2.0,-15.0,2.0
2,9,"T(4,9)",0,4.0,-16.0,4.0
3,11,"T(4,11)",0,4.0,-21.0,4.0
4,13,"T(4,13)",2,6.0,-26.0,6.0
5,15,"T(4,15)",0,6.0,-31.0,6.0
6,17,"T(4,17)",0,8.0,-32.0,8.0
7,19,"T(4,19)",0,8.0,-37.0,8.0
8,21,"T(4,21)",2,10.0,-42.0,10.0
9,23,"T(4,23)",0,10.0,-47.0,10.0
